In [ ]:
pip install pycryptodome tenseal

In [ ]:
import json
from Crypto.Random import get_random_bytes

def generate_keys():
    """Generate AES-256 encryption keys and save them in a file."""
    keys = {
        "aes_key": get_random_bytes(32).hex(),  # AES-256 Key
        "aes_iv": get_random_bytes(16).hex(),
    }

    with open("keys.json", "w") as key_file:
        json.dump(keys, key_file)

    print("🔑 AES Keys generated and saved in 'keys.json'")

generate_keys()


🔑 AES Keys generated and saved in 'keys.json'


In [ ]:
import json
import base64
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

class AESEncryption:
    def __init__(self):
        """Load or Generate AES-256 encryption keys."""
        try:
            with open("keys.json", "r") as key_file:
                keys = json.load(key_file)
            self.key = bytes.fromhex(keys["aes_key"])
            self.iv = bytes.fromhex(keys["aes_iv"])
        except FileNotFoundError:
            # Generate keys if not found
            self.key = get_random_bytes(32)  # AES-256 Key
            self.iv = get_random_bytes(16)   # AES IV
            keys = {
                "aes_key": self.key.hex(),
                "aes_iv": self.iv.hex(),
            }
            with open("keys.json", "w") as key_file:
                json.dump(keys, key_file)
            print("🔑 New AES Keys generated and saved.")

    def pad(self, data):
        """PKCS7 padding for AES block size (16 bytes)."""
        padding_length = AES.block_size - (len(data) % AES.block_size)
        return data + bytes([padding_length] * padding_length)

    def encrypt_file(self, input_file):
        """Encrypt a TXT or PDF file using AES-256."""
        with open(input_file, "rb") as file:
            plaintext = file.read()

        cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        ciphertext = cipher.encrypt(self.pad(plaintext))

        return base64.b64encode(self.iv + ciphertext).decode()


aes_encryptor = AESEncryption()

txt_file = "text.txt"
pdf_file = "Document.pdf"

aes_encrypted_txt = aes_encryptor.encrypt_file(txt_file)
aes_encrypted_pdf = aes_encryptor.encrypt_file(pdf_file)

combined_encrypted_data = aes_encrypted_txt + "||" + aes_encrypted_pdf

with open("aes_encrypted_output.txt", "w") as enc_file:
    enc_file.write(combined_encrypted_data)

print(" AES Encryption Complete! Encrypted data saved in 'aes_encrypted_output.txt'")


✅ AES Encryption Complete! Encrypted data saved in 'aes_encrypted_output.txt'


In [ ]:
# read
with open("aes_encrypted_output.txt", "r") as file:
    aes_encrypted_data = file.read()

# Convertion of AES-encrypted Base64 text into numeric representation for FHE
numeric_encoded_data = [ord(char) for char in aes_encrypted_data]


In [ ]:
import tenseal as ts

class FHEEncryption:
    def __init__(self):
        """Initialize Fully Homomorphic Encryption and Save Context."""
        self.context = ts.context(
            ts.SCHEME_TYPE.BFV,
            poly_modulus_degree=16384,
            plain_modulus=786433,
            coeff_mod_bit_sizes=[60, 40, 40, 60, 60]
        )
        self.context.generate_galois_keys()
        self.context.generate_relin_keys()

        with open("fhe_context.bin", "wb") as ctx_file:
            ctx_file.write(self.context.serialize(save_secret_key=True))

        print("🔑 FHE Context (with Secret Key) Saved!")

    def encrypt_fhe_chunk(self, chunk):
        """Encrypt a single chunk using Fully Homomorphic Encryption (FHE)."""
        encrypted_fhe_data = ts.bfv_vector(self.context, chunk)
        return encrypted_fhe_data.serialize()

    def encrypt_fhe_in_chunks(self, data, chunk_size=3000):
        """Encrypt AES-encrypted data in chunks to fit into FHE ciphertext."""
        chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
        encrypted_chunks = [self.encrypt_fhe_chunk(chunk) for chunk in chunks]
        return encrypted_chunks

fhe_encryptor = FHEEncryption()

fhe_encrypted_chunks = fhe_encryptor.encrypt_fhe_in_chunks(numeric_encoded_data)

with open("final_encrypted_output.fhe", "wb") as fhe_file:
    for chunk in fhe_encrypted_chunks:
        fhe_file.write(len(chunk).to_bytes(4, 'big'))
        fhe_file.write(chunk)

print(" FHE Encryption Complete! Output saved in 'final_encrypted_output.fhe'")


🔑 FHE Context (with Secret Key) Saved!
✅ FHE Encryption Complete! Output saved in 'final_encrypted_output.fhe'


DECRYPTION

In [ ]:
import tenseal as ts

class FHEDecryption:
    def __init__(self):
        """Load Fully Homomorphic Encryption Context (with Secret Key)."""
        with open("fhe_context.bin", "rb") as ctx_file:
            self.context = ts.context_from(ctx_file.read())

    def decrypt_fhe_chunk(self, encrypted_chunk):
        """Decrypt a single chunk using Fully Homomorphic Encryption (FHE)."""
        encrypted_vector = ts.bfv_vector_from(self.context, encrypted_chunk)
        return encrypted_vector.decrypt()  #  secret key

    def decrypt_fhe_chunks(self, encrypted_file):
        """Decrypt all chunks stored in a file and reconstruct original text."""
        decrypted_data = []
        with open(encrypted_file, "rb") as file:
            while True:
                length_bytes = file.read(4)
                if not length_bytes:
                    break
                chunk_length = int.from_bytes(length_bytes, 'big')
                encrypted_chunk = file.read(chunk_length)
                decrypted_data.extend(self.decrypt_fhe_chunk(encrypted_chunk))
        return "".join(chr(num) for num in decrypted_data)

fhe_decryptor = FHEDecryption()

decrypted_aes_base64 = fhe_decryptor.decrypt_fhe_chunks("final_encrypted_output.fhe")

with open("decrypted_aes_output.txt", "w") as dec_file:
    dec_file.write(decrypted_aes_base64)

print(" FHE Decryption Complete! AES-encrypted data restored in 'decrypted_aes_output.txt'. Ready for AES decryption.")


✅ FHE Decryption Complete! AES-encrypted data restored in 'decrypted_aes_output.txt'. Ready for AES decryption.


In [ ]:
import json
import base64
from Crypto.Cipher import AES

class AESEncryption:
    def __init__(self):
        """Load AES-256 decryption keys from file."""
        with open("keys.json", "r") as key_file:
            keys = json.load(key_file)

        self.key = bytes.fromhex(keys["aes_key"])
        self.iv = bytes.fromhex(keys["aes_iv"])

    def unpad(self, data):
        """Remove PKCS7 padding safely."""
        padding_length = data[-1]
        if padding_length < 1 or padding_length > AES.block_size:
            return data
        return data[:-padding_length]

    def decrypt_base64(self, encrypted_base64):
        """Decrypt AES-256 Base64 encoded data."""
        encrypted_data = base64.b64decode(encrypted_base64)
        iv = encrypted_data[:16]
        ciphertext = encrypted_data[16:]  # Extract

        if len(ciphertext) % AES.block_size != 0:
            raise ValueError("⚠️ AES data is corrupted or improperly padded!")

        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        decrypted_data = self.unpad(cipher.decrypt(ciphertext))

        return decrypted_data

aes_decryptor = AESEncryption()


In [ ]:
with open("decrypted_aes_output.txt", "r") as file:
    decrypted_aes_base64 = file.read()

if len(decrypted_aes_base64) % 16 != 0:
    print("⚠️ Warning: AES data might be incorrectly retrieved from FHE!")

#  Split
aes_encrypted_txt, aes_encrypted_pdf = decrypted_aes_base64.split("||")

# Decrypt the original TXT file
original_txt_data = aes_decryptor.decrypt_base64(aes_encrypted_txt)
with open("decrypted_document.txt", "wb") as txt_file:
    txt_file.write(original_txt_data)

# Decrypt the original PDF file
original_pdf_data = aes_decryptor.decrypt_base64(aes_encrypted_pdf)
with open("decrypted_certificate.pdf", "wb") as pdf_file:
    pdf_file.write(original_pdf_data)

print("Decryption Complete! Original files restored:")
print("Decrypted TXT saved as: decrypted_document.txt")
print("Decrypted PDF saved as: decrypted_certificate.pdf")


⚠️ Warning: AES data might be incorrectly retrieved from FHE!
✅ Decryption Complete! Original files restored:
📄 Decrypted TXT saved as: decrypted_document.txt
📜 Decrypted PDF saved as: decrypted_certificate.pdf
